<a href="https://colab.research.google.com/github/abhijit-z/IDS_506_Omicron_Tweets_Analysis/blob/main/Further_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import os
import re
import string
import nltk

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#px = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/IndivFiles/df_ind_11292021.csv"
#dfpx = pd.read_csv(px, engine='python')

In [ ]:
'''
print(len(dfpx))
print(len(dfpx['id'].unique()))
print(len(dfpx['user_id'].unique()))
'''

"\nprint(len(dfpx))\nprint(len(dfpx['id'].unique()))\nprint(len(dfpx['user_id'].unique()))\n"

In [ ]:
uniq_list = ['news', 'agency', 'official', 'government', 'group', 'media', 'organisation', 'organization', 
             'alliance', '24/7', '24x7','24X7', 'livecast', 'commission', 'business', 'breaking', 'company', 
              'network', 'latest', 'suntimes', 'bostonglobe', 'nwitimes', 'daily',
             'corporation', 'corp' 'inc', 'ltd', 'newsmagazine', 'institution', 'public health', 'times', 'moneycontrol',
             'pib', 'bloomberg', 'world news', 'times']

date1 = '2021-11-21'
date2 = '2022-01-31'

#date1 = datetime.datetime.strptime('12012021', '%m%d%Y')
#date2 = datetime.datetime.strptime('12022021', '%m%d%Y')

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
print(mydates)
for d in pd.date_range(date1, date2, freq='d'):
  d_str = d.strftime("%m%d%Y")
  print(d_str)
  path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter1_res/IndivFiles/df_ind_" + d_str + ".csv"

  df = pd.read_csv(path, engine= 'python')
  df_reduced = df[['coordinates', 'created_at', 'hashtags', 'place', 
                   'source','user_id', 'user_description', 'user_name', 'id',
       'user_screen_name', 'user_urls', 'user_verified', 'text', 'ind_org']]

  nltk.download('punkt')
  nltk.download('stopwords')

  STOP_WORDS = stopwords.words()

  # removing the emojies
  # https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis
  EMOJI_PATTERN = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)


  def cleaning(text):
      """
      Convert to lowercase.
      Rremove URL links, special characters and punctuation.
      Tokenize and remove stop words.
      """
      text = text.lower()
      text = re.sub('https?://\S+|www\.\S+', '', text)
      text = re.sub('@[A-Za-z0-9]+', '', text)
      text = re.sub('<.*?>+', '', text)   # replacing HTML text with blank or whitespace
      text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
      text = re.sub('\n', '', text)
      text = re.sub('[’“”…]', '', text)
      
      text = text.encode("ascii", "ignore")     # consider only english alphanumeric characters
      text = text.decode()
      
      text = EMOJI_PATTERN.sub(r'', text)
    
      
      
      # removing the stop-words
      text_tokens = word_tokenize(text)
      tokens_without_sw = [
        word for word in text_tokens if not word in STOP_WORDS]
      filtered_sentence = (" ").join(tokens_without_sw)
      text = filtered_sentence
      
      return text

  def func2(val1, val2):
    for word in uniq_list:
        if (word in val1):     # may use val2 ie user_verified status for additional filtering
            return "org"
        else:
            return "ind"

  #if __name__ == "__main__":
  #    max_rows = 1000  # 'None' to read whole file
      
  #df = pd.read_csv("Labeled_tweets.csv")
  #df.head()

 
  df_reduced['name_plus_desp'] = df_reduced['user_name'] + " " + df_reduced['user_description']
  df_reduced['name_plus_desp'] = df_reduced['name_plus_desp'].astype(str)
  df_reduced.head() 
  
  #dt = df_reduced['name_plus_desp'].apply(cleaning)
  df_reduced['name_plus_desp_clean'] = df_reduced['name_plus_desp'].apply(cleaning)

  
  #print(df_reduced['name_plus_desp_clean'])
  print("\n")
  
  # word_count = Counter(" ".join(dt).split()).most_common(10)
  # word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
  # print(word_frequency)

  df_reduced['new_col2'] = df_reduced.apply(lambda row: func2(row['name_plus_desp_clean'], row['user_verified']), axis =1)
  print(df_reduced['new_col2'].value_counts())
  
  path2 = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter2_res/Further_filtering"

  #os.makedirs(path, exist_ok=False)
  isExist = os.path.exists(path2)
  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path2)
    print("The new directory is created!")
  dest_path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter2_res/Further_filtering/df_ff_" + d_str + ".csv"
  df_reduced.to_csv(dest_path)


DatetimeIndex(['2021-11-21', '2021-11-22', '2021-11-23', '2021-11-24',
               '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28',
               '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02',
               '2021-12-03', '2021-12-04', '2021-12-05', '2021-12-06',
               '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10',
               '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-14',
               '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18',
               '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22',
               '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26',
               '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30',
               '2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03',
               '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
               '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11',
               '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15',
      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ind    41821
org     2378
Name: new_col2, dtype: int64
The new directory is created!
11222021
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ind    51431
org     3547
Name: new_col2, dtype: int64
11232021
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ind    47425
org     3411
Name: new_col2, dtype: int64
11242021
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ind    49540
org     3127
Name: new_col2, dtype: int64
11252021
[nltk_data] Downlo

In [ ]:
#p = "/content/drive/MyDrive/IDS 506 Healthcare Analytics/506 Project/Further Filtered/df_ff_12222021.csv"
#df_p = pd.read_csv(p, engine='python')

In [ ]:
#df_p.head(100)

In [ ]:

'''

def func2(val1, val2):
    for word in uniq_list:
        if (word in val1):     # may use val2 ie user_verified status for additional filtering
            return "org"
        else:
            return "ind"

df_reduced['new_col2'] = df_reduced.apply(lambda row: func2(row['name_plus_desp_clean'], row['user_verified']), axis =1)
'''

'\n\ndef func2(val1, val2):\n    for word in uniq_list:\n        if (word in val1):     # may use val2 ie user_verified status for additional filtering\n            return "org"\n        else:\n            return "ind"\n\ndf_reduced[\'new_col2\'] = df_reduced.apply(lambda row: func2(row[\'name_plus_desp_clean\'], row[\'user_verified\']), axis =1)\n'